In [6]:
import tensorflow as tf
from tensorflow.keras import layers

In [7]:
print(tf.__version__)

2.11.0


In [8]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

<h3>tensorflow 2.11 doesn't work by using GPU!! ㅠㅠ on Windows</h3>

In [2]:
loc = "./IMG_CLASSES"

<h3> I made new directories for 5 Diseases</h3>

In [2]:
train_dir = "./output/train2"
test_dir =  "./output/test2"
val_dir = "./output/val2"

In [9]:
from tensorflow.keras.preprocessing import image_dataset_from_directory


train_data = image_dataset_from_directory(train_dir,label_mode = "categorical",
                                          image_size = (224,224),batch_size = 32,
                                         shuffle = True,seed = 42)
test_data = image_dataset_from_directory(test_dir,label_mode = "categorical",
                                          image_size = (224,224),batch_size = 32,
                                         shuffle = False,seed = 42)
val_data = image_dataset_from_directory(val_dir,label_mode = "categorical",
                                          image_size = (224,224),batch_size = 32,
                                         shuffle = False,seed = 42)

Found 8163 files belonging to 5 classes.
Found 1026 files belonging to 5 classes.
Found 1018 files belonging to 5 classes.


In [11]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor = "val_loss",patience = 6,
                                             min_delta = 0.0001)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = "val_loss",factor = 0.2,
                                                patience = 4,min_lr = 1e-7)

In [12]:
model = tf.keras.applications.EfficientNetB7(include_top = False)
model.trainable = False
#To "freeze" a layer means to exclude it from training, i.e. its weights will never be updated.

In [13]:
from tensorflow.keras.layers.experimental import preprocessing

data_aug = tf.keras.Sequential([
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomFlip("horizontal")
],name = "data_augmentation_layer")

In [14]:
inputs = layers.Input(shape = (224,224,3),name = "input_layer")
x = data_aug(inputs)
x = model(x)
x = layers.GlobalAvgPool2D(name = "pooling_layer")(x)
# parameter used to specify the size of the output generated from the layer.
x = layers.Dense(32,activation = "relu",kernel_initializer = tf.keras.initializers.he_normal())(x)
x = layers.Dense(5)(x)
outputs = layers.Activation("softmax",dtype = tf.float32)(x)
model = tf.keras.Model(inputs,outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation_layer (Se  (None, None, None, 3)    0         
 quential)                                                       
                                                                 
 efficientnetb7 (Functional)  (None, None, None, 2560)  64097687 
                                                                 
 pooling_layer (GlobalAverag  (None, 2560)             0         
 ePooling2D)                                                     
                                                                 
 dense (Dense)               (None, 32)                81952     
                                                                 
 dense_1 (Dense)             (None, 5)                 165   

In [15]:
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),optimizer = tf.keras.optimizers.Adam(),metrics = ["accuracy"])

In [16]:
history_1 = model.fit(train_data,epochs =20 , validation_data = val_data, 
                      callbacks = [early_stop,reduce_lr])

Epoch 1/20
256/256 [==============================] - 944s 4s/step - loss: 0.7907 - accuracy: 0.6778 - val_loss: 0.7077 - val_accuracy: 0.7063 - lr: 0.0010
Epoch 2/20
256/256 [==============================] - 990s 4s/step - loss: 0.6629 - accuracy: 0.7312 - val_loss: 0.6789 - val_accuracy: 0.7348 - lr: 0.0010
Epoch 3/20
256/256 [==============================] - 1005s 4s/step - loss: 0.6158 - accuracy: 0.7509 - val_loss: 0.6614 - val_accuracy: 0.7338 - lr: 0.0010
Epoch 4/20
256/256 [==============================] - 984s 4s/step - loss: 0.5848 - accuracy: 0.7685 - val_loss: 0.6442 - val_accuracy: 0.7485 - lr: 0.0010
Epoch 5/20
256/256 [==============================] - 1001s 4s/step - loss: 0.5669 - accuracy: 0.7774 - val_loss: 0.6225 - val_accuracy: 0.7505 - lr: 0.0010
Epoch 6/20
256/256 [==============================] - 967s 4s/step - loss: 0.5414 - accuracy: 0.7876 - val_loss: 0.6253 - val_accuracy: 0.7603 - lr: 0.0010
Epoch 7/20
256/256 [==============================] - 972s 4s/

In [17]:
print("Testing Accuracy",model.evaluate(test_data)) 

33/33 [==============================] - 108s 3s/step - loss: 0.5888 - accuracy: 0.7865
Testing Accuracy [0.588804304599762, 0.7865496873855591]


In [18]:
#model.save('my_model')
tf.saved_model.save(
    model, 'export_dir2', signatures=None, options=None
)

INFO:tensorflow:Assets written to: export_dir2\assets


INFO:tensorflow:Assets written to: export_dir2\assets


In [19]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# FP16 양자화 설정
converter.target_spec.supported_types = [tf.float16]
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# 모델 양자화
tflite_model = converter.convert()

# 변환된 모델을 .tflite 파일에 저장
open("best_model_5c_2.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\admin\AppData\Local\Temp\tmpon9qoaw6\assets


INFO:tensorflow:Assets written to: C:\Users\admin\AppData\Local\Temp\tmpon9qoaw6\assets


127813992

<h2> Evaluate Model</h2>

In [25]:
tp = model.predict(test_data)

33/33 [==============================] - 662s 20s/step


In [27]:
tpC = tp.argmax(axis=-1)

In [37]:
import numpy as np
y = np.concatenate([y for x, y in test_data], axis=0)

In [40]:
y = y.argmax(axis=-1)

In [43]:
from sklearn.metrics import classification_report
print(classification_report(tpC,y))

              precision    recall  f1-score   support

           0       0.72      0.67      0.69       183
           1       0.64      0.72      0.68       188
           2       0.61      0.66      0.63       119
           3       1.00      0.98      0.99       339
           4       0.75      0.71      0.73       197

    accuracy                           0.79      1026
   macro avg       0.74      0.75      0.74      1026
weighted avg       0.79      0.79      0.79      1026

